In [1]:
import xarray as xr
import pandas as pd
import numpy as np

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
import rioxarray

In [3]:
ds = xr.open_dataset("TS_annual_means.nc")

In [4]:
ds

<xarray.Dataset>
Dimensions:  (lat: 192, lon: 288, year: 95)
Coordinates:
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * year     (year) int64 2006 2007 2008 2009 2010 ... 2096 2097 2098 2099 2100
Data variables:
    TS       (year, lat, lon) float32 ...

In [8]:
ds["TS"]

<xarray.DataArray 'TS' (year: 95, lat: 180, lon: 288)>
array([[[-28.298508 , -27.96428  , -27.63881  , ..., -29.634644 ,
         -29.0914   , -28.651505 ],
        [-27.695938 , -27.890503 , -27.887543 , ..., -27.098679 ,
         -27.256042 , -27.447601 ],
        [-31.113617 , -31.22055  , -31.308655 , ..., -30.886734 ,
         -30.893845 , -31.011047 ],
        ...,
        [-19.782883 , -19.813126 , -19.846817 , ..., -19.708496 ,
         -19.727478 , -19.75319  ],
        [-19.948334 , -19.963928 , -19.983246 , ..., -19.90512  ,
         -19.926163 , -19.938568 ],
        [-20.112198 , -20.139984 , -20.169525 , ..., -20.067749 ,
         -20.077805 , -20.089294 ]],

       [[-28.103775 , -27.723557 , -27.37004  , ..., -29.448212 ,
         -28.909912 , -28.445679 ],
        [-27.46074  , -27.609665 , -27.649704 , ..., -26.904099 ,
         -27.045013 , -27.186584 ],
        [-30.785004 , -30.894424 , -30.962296 , ..., -30.61615  ,
         -30.624603 , -30.721573 ],
...
        [ -3.3721008,  -3.3752441,  -3.3774414, ...,  -3.3889465,
          -3.3790894,  -3.3710938],
        [ -3.6096802,  -3.6030579,  -3.5976562, ...,  -3.6339722,
          -3.6290283,  -3.6211243],
        [ -3.7937317,  -3.794159 ,  -3.8012085, ...,  -3.808014 ,
          -3.8017883,  -3.7933044]],

       [[-23.232315 , -22.762024 , -22.290619 , ..., -24.696869 ,
         -24.122116 , -23.64943  ],
        [-21.915283 , -21.972778 , -21.985046 , ..., -21.633224 ,
         -21.694336 , -21.763199 ],
        [-24.621918 , -24.732422 , -24.873016 , ..., -24.387787 ,
         -24.466614 , -24.552338 ],
        ...,
        [ -3.123291 ,  -3.1321106,  -3.1387634, ...,  -3.1247864,
          -3.118988 ,  -3.1160583],
        [ -3.4062805,  -3.4000854,  -3.3952026, ...,  -3.4236145,
          -3.4230957,  -3.4169922],
        [ -3.6261292,  -3.6269836,  -3.6357727, ...,  -3.6346436,
          -3.6305237,  -3.6243591]]], dtype=float32)
Coordinates:
  * lat          (lat) float64 -84.35 -83.4 -82.46 -81.52 ... 82.46 83.4 84.35
  * lon          (lon) float64 -180.0 -178.8 -177.5 -176.2 ... 176.2 177.5 178.8
  * year         (year) int64 2006 2007 2008 2009 2010 ... 2097 2098 2099 2100
    spatial_ref  int64 0

In [5]:
ds.coords['lon'] = (ds.coords['lon'] + 180) % 360 - 180
ds = ds.sortby(ds.lon)
ds["TS"] = ds["TS"] - 273.15

In [6]:
ds = ds.where(np.abs(ds.lat) < 85.06, drop=True)

ds.rio.set_spatial_dims("lon","lat")
ds.rio.write_crs(4326, inplace=True)

# Define the custom bounds for the Web Mercator projection
dst_bounds = (-20037508.342789244, -20037508.342789244, 20037508.342789244, 20037508.342789244)

# Reproject to Web Mercator using rioxarray
ds_3857 = ds.rio.reproject("EPSG:3857", resolution=None, bounds=dst_bounds)

# Save the output NetCDF file
ds_3857.to_netcdf('TS_annual_reproject.nc')

In [7]:
ds_3857

<xarray.Dataset>
Dimensions:      (x: 242, y: 238, year: 95)
Coordinates:
  * x            (x) float64 -1.995e+07 -1.979e+07 ... 1.978e+07 1.995e+07
  * y            (y) float64 1.966e+07 1.949e+07 ... -1.943e+07 -1.96e+07
  * year         (year) int64 2006 2007 2008 2009 2010 ... 2097 2098 2099 2100
    spatial_ref  int64 0
Data variables:
    TS           (year, y, x) float32 -20.14 -20.17 -20.2 ... -24.12 -23.65

In [19]:
dff = ds_3857['TS'].sel(x=-1.995e+07, y=1.966e+07, method="nearest")
dff = dff.to_dataframe().reset_index()[['year', 'TS']]
dff

,year,TS
0,2006,-20.139984
1,2007,-20.102020
2,2008,-19.688889
3,2009,-19.626892
4,2010,-19.795258
...,...,...
90,2096,-4.482086
91,2097,-4.184662
92,2098,-3.993317
93,2099,-3.794159
